In [1]:
%reload_ext autoreload
%autoreload 2

import os
from QC.inference_qc import _inference_main
from argparse import Namespace
from scipy.special import softmax
import pandas as pd
from glob import glob
import numpy as np
from tqdm import tqdm_notebook as tqdm

def get_dicom_dirs(base_dir, override=False):
    for fname in glob('{}/**/*.dcm'.format(base_dir), recursive=True):
        base_dir = '/'.join(fname.split('/')[:-2])
        break

    dirs = os.listdir(base_dir)
    dirs = [d for d in dirs if '.DS_Store' not in d]
    if len(dirs) > 3:
        kw_exc = ['t2', 'flair']

        dirs_filt = []
        for d in dirs:
            kw_found = False
            for kw in kw_exc:
                kw_found = (kw in d.lower())
                if kw_found: break

            if not kw_found:
                dirs_filt.append(d)

        dirs = dirs_filt[:3]

    dirs_split = np.array([d.split('_') for d in dirs], dtype=object)
    try:
        dirs_sorted = np.array(dirs)[np.argsort([int(dd[0]) for dd in dirs_split])]
    except:
        dirs_sorted = np.array(dirs)[np.argsort([int(dd[-1]) for dd in dirs_split])]

    dir_list = []

    for d in dirs_sorted:
        if override or 'ax' in d.lower() or 'sag' in d.lower():
            if override or 'mprage' in d.lower() or 'bravo' in d.lower():
                if override or 'reformat' not in d.lower():
                    dir_list.append(os.path.join(base_dir, d))
        if len(dir_list) == 3:
            break

    return dir_list

def get_qc_probs(fpaths, fpath_model='/home/srivathsa/projects/QualityControl/QC/QCapp/models'):
    args = Namespace(i=fpaths, m=fpath_model, c=None, o=None, overwrite=True, error_path='/tmp')
    qc_results = _inference_main(args, save=False, output_pooled=True)
    sm_results = [softmax(x, axis=-1) for x in qc_results]
    probs = [x[0][1] for x in sm_results]
    return probs

/mnt/datasets/srivathsa/projects/QualityControl/QC/util/io_torch.py:14: UserWarning: cannot import ITK
  warnings.warn('cannot import ITK')
/home/srivathsa/projects/SubtleGad/train/subtle/utils/hyperparameter.py:8: UserWarning: Module test_tube not found - hyperparameter related functions cannot be used
  warnings.warn('Module test_tube not found - hyperparameter related functions cannot be used')
/mnt/datasets/srivathsa/projects/QualityControl/QC/QCapp/infer.py:28: UserWarning: cannot import subtle-train-utils
  warnings.warn('cannot import subtle-train-utils')


In [2]:
bad_cases = ["Patient_0120","Patient_0123","Patient_0149","Patient_0198","Patient_0241","Patient_0262",
             "Patient_0268","Patient_0281","Patient_0284","Patient_0330","Patient_0390","Patient_0454",
             "Patient_0517","Patient_0523","Patient_0540","Patient_0566","Patient_0171","Patient_0286",
             "Patient_0444","Patient_0285","Patient_0384","Patient_0577","Patient_0146","Patient_0175",
             "Patient_0193", "NO28"
]

good_cases = ["Patient_0118", "Patient_0119", "Patient_0251", "Patient_0261", "Patient_0565", "Patient_0432", 
              "Patient_0433", "Patient_0435", "Patient_0168", "Patient_0272", "NO31", "NO42", "NO120", 
              "NO75", "NO101", "NO111", "NO64", "NO43", "NO49", "NO55", "Prisma11", "Prisma18", "M001", "M003",
             "101_Id_061", "Id0032"]

In [3]:
df_inf = pd.read_csv('/home/srivathsa/projects/studies/gad/gad_inf_repo.csv')
df_inf = df_inf.rename(columns={"Inference Path (dgx-1)": "Inference_Path"})
inf_list = df_inf.to_dict(orient='records')

data_list = []

for cnum in bad_cases:
    rec = [d for d in inf_list if d['Case'] == cnum][0]
    if 'LOCAL' in rec['Inference_Path']:
        rec['Inference_Path'] = '/home/srivathsa/projects/studies/gad/stanford/data/aa5daa_010c89'
        
    data_list.append({
        'case': cnum,
        'artifacts': 'yes',
        'data_path': '{}/{}'.format('/'.join(rec['Inference_Path'].split('/')[:-1]), cnum)
    })
    
for cnum in good_cases:
    rec = [d for d in inf_list if d['Case'] == cnum][0]
    if 'LOCAL' in rec['Inference_Path']:
        rec['Inference_Path'] = '/home/srivathsa/projects/studies/gad/stanford/data/aa5daa_010c89'
    
    data_list.append({
        'case': cnum,
        'artifacts': 'no',
        'data_path': '{}/{}'.format('/'.join(rec['Inference_Path'].split('/')[:-1]), cnum)
    })

In [4]:
for data_row in tqdm(data_list, total=len(data_list)):
    fpath_pre, fpath_low, _ = get_dicom_dirs(data_row['data_path'], override=True)
    
    pre_qc, low_qc = get_qc_probs([fpath_pre, fpath_low])
    
    data_row['pre_qc'] = pre_qc
    data_row['low_qc'] = low_qc

/home/srivathsa/miniconda3/envs/qcapp/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/52 [00:00<?, ?it/s]

loaded  /home/srivathsa/projects/QualityControl/QC/QCapp/models/checkpoint_1.pt
0 of 2
loading time =  5.905690670013428  s
inference time =  0.017314910888671875  s
1 of 2
loading time =  5.965472936630249  s
inference time =  0.012696504592895508  s
loaded  /home/srivathsa/projects/QualityControl/QC/QCapp/models/checkpoint_1.pt
0 of 2
loading time =  5.303775310516357  s
inference time =  0.009942293167114258  s
1 of 2
loading time =  5.840364933013916  s
inference time =  0.011349916458129883  s
loaded  /home/srivathsa/projects/QualityControl/QC/QCapp/models/checkpoint_1.pt
0 of 2
loading time =  5.842448711395264  s
inference time =  0.013196468353271484  s
1 of 2
loading time =  4.588598251342773  s
inference time =  0.010402917861938477  s
loaded  /home/srivathsa/projects/QualityControl/QC/QCapp/models/checkpoint_1.pt
0 of 2
loading time =  5.930089235305786  s
inference time =  0.007638454437255859  s
1 of 2
loading time =  5.8819661140441895  s
inference time =  0.0115547180175

loading time =  3.2327749729156494  s
inference time =  0.0075511932373046875  s
loaded  /home/srivathsa/projects/QualityControl/QC/QCapp/models/checkpoint_1.pt
0 of 2
loading time =  2.8114702701568604  s
inference time =  0.008862733840942383  s
1 of 2
loading time =  2.6792805194854736  s
inference time =  0.006827592849731445  s
loaded  /home/srivathsa/projects/QualityControl/QC/QCapp/models/checkpoint_1.pt
0 of 2
loading time =  2.7827534675598145  s
inference time =  0.01138162612915039  s
1 of 2
loading time =  2.9606857299804688  s
inference time =  0.007759809494018555  s
loaded  /home/srivathsa/projects/QualityControl/QC/QCapp/models/checkpoint_1.pt
0 of 2
loading time =  5.529508590698242  s
inference time =  0.007421970367431641  s
1 of 2
loading time =  11.428718090057373  s
inference time =  0.0076220035552978516  s
loaded  /home/srivathsa/projects/QualityControl/QC/QCapp/models/checkpoint_1.pt
0 of 2
loading time =  2.9150960445404053  s
inference time =  0.0064461231231

In [6]:
df_qc_scores = pd.DataFrame(data_list)
df_qc_scores

,case,artifacts,data_path,pre_qc,low_qc
0,Patient_0120,yes,/home/srivathsa/projects/studies/gad/stanford/...,0.910511,0.924420
1,Patient_0123,yes,/home/srivathsa/projects/studies/gad/stanford/...,0.018396,0.744418
2,Patient_0149,yes,/home/srivathsa/projects/studies/gad/stanford/...,0.000893,1.000000
3,Patient_0198,yes,/home/srivathsa/projects/studies/gad/stanford/...,0.998262,1.000000
4,Patient_0241,yes,/home/srivathsa/projects/studies/gad/stanford/...,0.976523,0.955898
5,Patient_0262,yes,/home/srivathsa/projects/studies/gad/stanford/...,0.372309,0.022329
6,Patient_0268,yes,/home/srivathsa/projects/studies/gad/stanford/...,0.980767,0.994458
7,Patient_0281,yes,/home/srivathsa/projects/studies/gad/stanford/...,0.001776,0.000681
8,Patient_0284,yes,/home/srivathsa/projects/studies/gad/stanford/...,0.002563,0.999998
9,Patient_0330,yes,/home/srivathsa/projects/studies/gad/stanford/...,1.000000,1.000000


In [7]:
flist = get_dicom_dirs('/home/srivathsa/projects/studies/gad/stanford/data/Patient_0171')

In [8]:
flist

['/home/srivathsa/projects/studies/gad/stanford/data/Patient_0171/7_AX_BRAVO',
 '/home/srivathsa/projects/studies/gad/stanford/data/Patient_0171/10_AX_BRAVO_+C',
 '/home/srivathsa/projects/studies/gad/stanford/data/Patient_0171/14_AX_BRAVO_+C']